In [14]:
include("polynomials.jl")
include("quotientrings.jl")

Base.show(io::IO, p::Polynomial{Quot{T, Q}}) where {T <: Number, Q <: Val} = 
print(io, join(reverse!(["$(p[i].a) 𝕩^$i" for i ∈ 0:p.degree]), " + ") * "    mod $(getval(Q))")

# SECURE MULTIPARTY COMPUTATION: AN OVERVIEW

*Group Members:* Anand Rao Tadipatri, Anirudh Rachuri, Hrishikesh V, Kapil Chandak, Srijay Sutar

# What is Secure MPC?

# Some motivating examples

So to just get some idea of where MPC is really used we start with some examples.

## Bank locker

> "In response to a recent RTI, the RBI and 19 PSU banks explained that “the relationship they have with customers with regard to lockers is that of lessee (tenant) and lessor (landlord)”, and no compensation will be given for theft or burglary of valuables in safe deposit boxes of public sector banks as the locker hiring agreement absolves them of all liability"  -TOI

So due to this response and also otherwise we want to understand about the security of the things which we keep in our bank lockers.For the entire details of the issues look at the link which is provided in reference in order to understand various issues associated with it.

MPC and related ideas helps us to understand about the security aspects about the safety of the things which we keep in our bank lockers.

## Private contact discovery

-------Google adds example which Aniruddha talked about . Will add the exact points after todays meeting------

# Security Notions in MPC
(Ideal-Real Simulation paradigm)

# What is Secret Sharing?
Secret sharing (also called secret splitting) refers to methods for distributing a secret among a group of participants, each of whom is allocated a share of the secret. The secret can be reconstructed only when a sufficient number, of possibly different types, of shares are combined together; individual shares are of no use on their own.

Secret sharing is quite an old cryptographic method, with existing real-world applications, for instance in Bitcoin signatures.
Other than from a purely conceptual point of view, secret sharing is also intriguing to study from a performance point of view, as it relies on a bare minimum of cryptographic assumptions.
<br /> <br />

<center>
<figure>
    <img src="secret.jpg"  height="500" width="500"/>
    
</figure>
</center>

## Shamir Secret Sharing
Shamir's Secret Sharing (SSS) is used to secure a secret in a distributed way, most often to secure other encryption keys. The secret is split into multiple parts, called **shares**. These shares are used to reconstruct the original secret.

To unlock the secret via Shamir's secret sharing, you need a minimum number of shares. This is called the **threshold**, and is used to denote the minimum number of shares needed to unlock the secret.
<br /><br />

<center>
<figure>
    <img src="sss.png"  height="350" width="350"/>
   
</figure>
</center>

### General Protocol
Formulated by Adi Shamir, this was one of the first secret sharing schemes. We explain a geometric version of secret sharing, which is based on polynomial interpolation over finite fields.



Assume that a secret ($s$) is to be shared among $n$ people. The protocol is as follows:
* A threshold $t(≤ n)$ is chosen.
* A polynomial passing through $(0, s)$ of degree $t-1$ is chosen.
* $n$ distinct points lying on the polynomial (called shares) are selected.
* A point on the polynomial $(x_i, y_i), 1≤i≤t $ is shared with each of the $n$ people.
* If $t$ people combine, the polynomial and hence the secret can be determined.
It is not possible to determine the polynomial unless atleast $t$ people come together. This is due to the fact that there are infinite polynomials passing through $(t-k)$ points for any $k \in [1, t-1]$.



### Usecase: Computing Linear Combination of inputs

# An application of Shamir Secret Sharing to computing sums

Suppose $n$ people with secret numbers $s_1, s_2, \ldots, s_n$ want to compute the sum of their secret numbers without revealing the individual numbers.


This can be accomplished through a modification of the Shamir Secret Sharing (SSS) protocol. Another variant of this approach yields a method for computing the product of the secret numbers, and consequently, any polynomial function of the variables $s_1, s_2, \ldots, s_n.$

The assumptions are:
- There is a secret communication channel between each pair of players
- A majority of the players are honest
- No player deviates from the protocol

In [ ]:
mutable struct Player{T} <: Any where {T <: Number}
    name::Symbol
    number::Int
    
    secret::T
    secretpoly::Polynomial{T}
    shares::Vector{T}
    
    computedvalue::T
end

In [ ]:
Base.show(io::IO, p::Player) = print(io, "Player $(p.number): $(p.name)")

In [ ]:
function randompolynomial(D; degree) 
    Polynomial([D; rand(typeof(D), degree)])
end

In [ ]:
Player(name::Symbol, number::Int; num_players = N, threshold = t, 𝔽ield = 𝔽) = begin
    Player(name, number, rand(𝔽ield), Polynomial([zero(𝔽ield)]), zeros(𝔽ield, num_players), zero(𝔽ield))
end

**The set-up:**
A group of $n$ people, with secret numbers $s_1, s_2, \ldots, s_n$ respectively, want to securely compute $(t, n)$ secret shares for the sum of their numbers.

**Implementation:**
Consider a set-up with $5$ people performing a $(2, 5)$ secret sharing with secret numbers drawn from the prime field $\mathbb{Z}_{17}$.

In [ ]:
𝔽 = Quot{Int, Val{17}};  #the field
N, t = 5, 2;  #the number of players and the threshold

In [ ]:
Anand = Player(:Anand, 1)
Anirudh = Player(:Anirudh, 2)
Hrishikesh = Player(:Hrishikesh, 3)
Kapil = Player(:Kapil, 4)
Srijay = Player(:Srijay, 5)

players = (Anand, Anirudh, Hrishikesh, Kapil, Srijay)

#### Step 1: Input sharing

Every player executes the $(t, n)$ Shamir Secret Sharing protocol with their secrets.

This differs from the regular protocol in two important ways:
- Each player receives a share of their own secret
- Instead of evaluating the secret polynomials at random inputs, the $i$th player is always given the share corresponding to the input $i$

In [ ]:
for p ∈ players;        p.secretpoly = randompolynomial(p.secret; degree = t);     end

In [ ]:
for p ∈ players, q ∈ players;     q.shares[p.number] = p.secretpoly(q.number);     end

#### Step 2: Sum evaluation

Each player then computes the sum of the shares they have received from the other players. The shares act like proxies for the values of the inputs.

The sum of the shares received by the $i$th player is

$$
\sum_{p \in players} p.secretpoly(i)
$$

Consider a new polynomial

$$
Q(x) = \sum_{p \in players} p.secretpoly(x)
$$

This is a degree $t$ polynomial with constant coefficient equal to 

$$
Q(0) = \sum_{p \in players} p.secretpoly(0) = \sum_{p \in players} p.secret
$$

Although the players have no way of computing $Q(x)$ without revealing their secret polynomials, they can individually compute the values of the shares $Q(i)$ by summing the shares received from the other players (as mentioned above).

In [ ]:
for p ∈ players;     p.computedvalue = sum(p.shares);  end

#### Step 3: Output construction

Now that the players have all computed their individual shares of the $t$-degree polynomial $Q(x)$, they can find the secret $Q(0)$ - which is the sum of their secret numbers - by combining any $t + 1$ shares and finding the interpolating polynomial.

In [ ]:
function interpolate(xs, ys::Vector{T}) where {T}
    𝕩 = Polynomial([zero(T), one(T)])
    
    #L(j) = prod([(𝕩 - Polynomial([xₖ])) for (k, xₖ) ∈ enumerate(xs) if k != j]) * 
    #Polynomial([inv(prod([(xs[j] - xₖ) for (k, xₖ) ∈ enumerate(xs) if k != j]))])
    
    L(j) = prod( [
            (𝕩 - Polynomial([xₖ])) * Polynomial([inv(xs[j] - xₖ)])
            for (k, xₖ) ∈ enumerate(xs) if k != j
            ] )
    
    sum([Polynomial([yⱼ]) * L(j) for (j, yⱼ) ∈ enumerate(ys)])
end       

interpolate(ps::Vector{Player{T}}) where {T <: Any} = interpolate(getfield.(ps, :number) .+ zero(T), getfield.(ps, :computedvalue))   

In [ ]:
interpolate([Anirudh, Kapil, Srijay])

In [ ]:
getfield.(players, :secret) |> sum

This approach can be generalised to compute

- Weighted sums of the secrets
- The product of a matrix (of suitable dimensions) with the vector of secrets
- Products of the secrets (after modifications)

and hence any polynomial function of the secret numbers $s_1, s_2, \ldots, s_n.$

---

# Private Set Intersection

## Introduction

Matching records is one of the most basic data analysis operations. There are many cases where data needs to be aligned across some common value — whether that’s joining between two different tables in a database or across two data sets stored in a file, or matching data sets between two separate entities.

## What is Private Set Intersection (PSI)?

PSI is a multi-party computation method using which 2 parties can share the intersection of the data that they have, without actually sharing their private raw data. An important and widely used variant is that of client-server systems. In this case, it is only the client who learns the intersection of the data sets. This is the more studied version, because using this variant in both directions is equivalent to the first variant.

PSI has its uses in areas like Plagiarism, Human genomic testing, Detection of tax evaders etc.


Protecting the confidentiality of a data set is a natural need and even a requisite in many scenarios. For example, when the data is user’s address book or the client genome of a genetic diagnosis service, such input data must be protected by cryptography.
<center>
<figure>
    <img src="psi.jpg"  height="400" width="400"/>
    <figcaption>Client-Server PSI</figcaption>
</figure>
</center>

## Working

### General PSI Protocol

#### Pseudorandom Functions

A pseudo-random function family (PRF) is a collection of efficiently computable functions with a key with the **property that outpus of the function on known inputs "look" completely random**. Thus for any given list of elements $x_1. x_2, x_3, \cdots x_n$, the series of values $F_k(x_1), F_k(x_2), \cdots F_k(x_n)$ look random. In particular, given $F_k(x_i)$, it is infeasible to determine the value of $x_i$ (here $F_k$ is the pseudorandom function and $k$ is the key).

#### Description of the protocol
In the following simple protocol, we utilise a tool called oblivious pseudorandom
function evaluation, which is a type of MPC protocol where the first party inputs $k$ and
the second party inputs $x$, and the second party receives $F_k(x)$ while the first party learns nothing
about $x$ (note that the second party learns $F_k(x)$ but nothing beyond that; in particular, $k$ remains
secret).

Now, consider two parties with respective sets of private elements; denote them $x_1, \cdots , x_n$ and
$y_1, \cdots , y_n$, respectively. Then the protocol proceeds as follows:
- The first party chooses a key $k$ for a pseudorandom function.
- The two parties run $n$ oblivious pseudorandom function evaluations: in the $i^{th}$ execution, the first party inputs $k$ and the second party inputs $y_i$. As a result, the second party learns $F_k(y_1),\cdots , F_k(y_n)$ while the first party learns nothing about $y_1,\cdots, y_n$.


- The first party locally computes $F_k(x_1),\cdots , F_k(x_n)$ and sends the list to the second party. It
can compute this since it knows $k$.
- The second party computes the intersection between the lists $F_k(y_1),\cdots , F_k(y_n)$ and $F_k(x_1),\cdots , F_k(x_n)$, and outputs all values $y_j$ for which $F_k(y_j)$ is in the intersection. (The party knows these values since it knows the association between $y_j$ and $F_k(y_j)$).



Note that the above protocol **reveals nothing but the intersection** since the first party learns nothing about $y_1,\cdots , y_n$ from the oblivious pseudorandom function evaluations, and the second party learns nothing about values of $x_j$ that are not in the intersection since the pseudorandom function hides the preimage values.

### PSI Based on Diffie-Hellman 

We can also modify the Diffie-Hellman Key Exchange protocol to use it for Private Set intersection. We will give a brief description of the protocol here.

#### Decisional Diffie-Hellman Assumption

Consider a (multiplicative) cyclic group $G$ of order $q$ ($q$ is prime), and with generator $g$. The DDH assumption states that, given $g^{a}$ and $g^{b}$ for uniformly and independently chosen $a, b \in \mathbb{Z}_q$, the value $g^{ab}$  "looks like" a random element in $G$.

#### Protocol
- Agree on a group $\mathbb{Z}_p$ with a generator $g$ ($p$ is prime). H is a pseudorandom function, which is also agreed upon. First party chooses secret $\alpha \in \mathbb{Z}_p$, second party chooses secret $\beta \in \mathbb{Z}_p$.
- For random $a,b,c$, one cannot distinguish $(g^a, g^b, g^{ab})$ from $g^a, g^b, g^c$ (DDH assumption)
- First party's dataset: $x_1, x_2,  \cdots x_n$, Second party's dataset: $y_1, y_2, \cdots y_n$. 
- First party computes $H(x_1)^\alpha, \cdots H(x_n)^\alpha$, second party computes $H(y_1)^\beta, \cdots H(y_n)^\beta$. 
- Both parties exchange their outputs. First party computes $(H(x_1)^\alpha)^\beta \cdots (H(x_n)^\alpha)^\beta$. Similarly, second party computes $(H(x_1)^\beta)^\alpha \cdots (H(x_n)^\beta)^\alpha$. 
- Since $(H(x)^\alpha)^\beta=(H(x)^\beta)^\alpha$, both parties can compare the 2 lists, and find the intersection. 

<center><img src="psidiffie.png"  height="500" width="500"/></center>

## Security

The generic PSI protocol satisfies the ideal-real simulation paradigm, as the protocol given leaks no information, apart from possibly leaking the size of the intersection. If an adversary intercepts the messages sent by the 2 parties, they can calculate the intersection set size, however they wouldn't have any information regarding the elements of the set. Neither party gains any information other than the intersection set itself. 

The Diffie Hellman PSI protocol has the added advantage that even the size of intersection set can not be calculated from intercepting the communication as the messages $H(x_1)^\alpha, \cdots H(x_n)^\alpha$ and $H(y_1)^\beta, \cdots H(y_n)^\beta$ cannot be decrypted without calculating $\alpha \cdot \beta$, which we are assuming cannot be calculated (DDH assumption).

## Applications

Now, let us look at some exemplary use cases where a PSI protocol is useful. Keep in mind the main traits of PSI: **two parties, both have interest in not showing their data to each other but need to find/do something with the elements in the intersection**:

Use Case |Description
-----|:-----
Private Contact Discovery|Finding out as a user (client) who of my private contacts also have a certain communication app (server)
DNA testing and pattern matching|The user gets their DNA sequenced and wants to find out about sequences linked to genetic diseases which are stored on a database (server).
Remote diagnostics|A medical diagnostic program assigns a status (sick or not sick with a certain disease) to a vectorized patient’s (client) electronic health record. While the client learns about her sickness, the program itself remains secret and the program owner (server) does not learn anything about the client’s data.
Advertising conversion|In order to compute accurate conversion rates from advertisements to actual purchases, the company and the ad providers compute the size of the intersection between customers who were shown the ads and customers who actually bought the product, without revealing any further information about the customers themselves. Here the ad providers are the server and the company are the clients.

# Conclusion

## Bank locker
We have seen the applications of PSI so now we re examine the bank locker problem we started with.

So our main issue is can my bank locker be opened without my key

### Assumptions:-

1.   We are talking of modern day bank lockers which require both keys the key of the owner of contents inside the locker and also of the branch manager and also that the lockers are digital.
2.   We are considering that the lockers are designed in such a way that it is not possible to acccess the things inside lockers by physically breaking the lockers.

We will use idea similar to Shamir secret sharing in order to understand this problem

We need to add 4 slides which will contain animation of the circle idea

# Some ongoing projects/places where MPC seems useful

## EzPC
https://youtu.be/-1H1Sp-_5YU

## EVM machines 

## Military documents

# The conclusion

So till now we saw with the help of some examples that how multi party computation works and where it might be useful.

Develponment of MPC is the need of the hour as this will also help in the developnment in data science,ML and AI in an appropriate manner.

MPC or similar ideas are also relevent in various security aspects and also in designing of varios different sorts of things which has a formal sense of security thereby helpings us doing various sorts of things in an efficient manner with the formal gaurantee that our data is not used by others without our consent.

---

# References

- https://eprint.iacr.org/2020/300.pdf (An Introduction to Secure Multiparty Computation by Yehuda Lindell)
- https://www.cs.princeton.edu/courses/archive/fall16/cos521/Lectures/lec21.pdf (An example of using Shamir's Secret Sharing scheme for addition and multiplication of secret numbers)
- https://timesofindia.indiatimes.com/blogs/toi-edit-page/security-of-my-bank-locker-here-are-simple-geometric-solutions-to-protect-valuables-or-confidential-information/(Times of India article explaining Security of bank lockers)
- https://main.sci.gov.in/supremecourt/2009/22857/22857_2009_39_1501_26314_Judgement_19-Feb-2021.pdf (Honorable Supreme court of India's judgement which contains references to several issues related to security of things in bank lockers)
- https://youtu.be/-1H1Sp-_5YU (EzPC)